# Globality Data Challenge

In the following exercise, I need to classify the 'case labels' of data in a given file.

# Load data

I begin by loading the data from the file

In [5]:
import math
import numpy as np
from random import *

In [6]:
fileName='consulting101.csv'

In [7]:
import csv
with open(fileName, 'rb') as csvfile:
    reader = csv.reader(csvfile)
    data=list(reader)

### The first line is the headers

In [8]:
data[0]

['', 'Case Type', 'Cousulting Ferm', 'Industry Coverage', 'text', 'title']

# Merging labels
Many of the case type labels are repetative so I will merge them using clustring and simhash (https://en.wikipedia.org/wiki/SimHash).

The clustering that I use is hierarchical clustering.

In [13]:
from simhash import Simhash
from sklearn.cluster import AgglomerativeClustering

Find all of the labels if there are multiples.

In [14]:
caseLabelAll=list(set([y for x in data[1:] 
                       for y in x[1].split(' | ')]))

Calculate the similarity matrix between labels.

In [15]:
connectivity=[
    [Simhash(x).distance(Simhash(y)) for x in caseLabelAll]
        for y in caseLabelAll]

Cluster the labels into 30.

In [16]:
n_clusters=30
model = AgglomerativeClustering(
        affinity='precomputed',
    linkage='average',
        n_clusters=n_clusters)
labels=model.fit_predict(connectivity)

Create a mapping between the labels and a numeric value.

In [17]:
caseToNum={caseLabelAll[j]: label for j,label in enumerate(labels) }

In [18]:
for x in range(30):
    for i,label in enumerate(labels):
        if label==x:
            print caseLabelAll[i]
    print 

math problem, brain teaser
quantitative case, math problem
math problem, quantitative case
math problem

finance
finance & accounting

new technology, new product
new technology
new product, new technology
new product

pricing and valuation
pricing
pricing, valuation
pricing & valuation
valuation

private equity (PE), investment
 investment
private equity, investment
private equity
private equity and investment
investment
private equity & investment
private equity (PE),  investment
investment, private equity

market entry, new market
market entry/new market
market entry

growth strategy
marketing strategy
strategy

operations strategy/marketing
operations strategy, supply chain optimization
operations strategy, business process optimization
 operations strategy
operations strategy & optimization
operations strategy, marketing
operations strategy, value chain optimization
operations strategy, optimization
operations strategy, value chain
operations strategy, outsourcing
business operati

# Merge companies

Similar to the label data, we need to consolidate the labels of the companies since they are often named different things but are the same company.

In [19]:
CousultingFermAll=list(set([y for x in data[1:] for y in x[2].split(' | ')]))

In [20]:
connectivity=[
    [Simhash(x).distance(Simhash(y)) for x in CousultingFermAll]
        for y in CousultingFermAll]

In [21]:
n_clusters=110
model = AgglomerativeClustering(
        affinity='precomputed',
    linkage='average',
        n_clusters=n_clusters)
labels=model.fit_predict(connectivity)

In [22]:
firmToNum={CousultingFermAll[j]: label for j,label in 
           enumerate(labels) }

In [ ]:
for x in range(125):
    for i,label in enumerate(labels):
        if label==x:
            print CousultingFermAll[i]
    print 

# Industry coverage

Similarly for the industry, we consolidate the labels.

In [23]:
indLabelAll=list(set([y for x in data[1:] 
                      for y in x[3].split(' | ')]))

In [24]:
connectivity=[
    [Simhash(x).distance(Simhash(y)) for x in indLabelAll]
        for y in indLabelAll]

In [25]:
n_clusters=100
model = AgglomerativeClustering(
        affinity='precomputed',
    linkage='average',
        n_clusters=n_clusters)
labels=model.fit_predict(connectivity)

In [ ]:
for x in range(125):
    for i,label in enumerate(labels):
        if label==x:
            print indLabelAll[i]
    print 

In [26]:
indToNum={indLabelAll[j]: label for j,label in enumerate(labels) }

# Text

To process the text, I first count the word usage.

In [27]:
from nltk import word_tokenize
from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
[stopWords.add(w) for w in ['!','#','$','%','&','(',')',',','.']];

I stem the word and remove stopwords.

In [28]:
allWords=[w2 for w2 in 
 [stemmer.stem(w) 
    for x in data[1:]
     for w in word_tokenize(x[4].decode('utf-8'))] 
          if w2 not in stopWords]

Count the word usage and remove unfrequent words

In [30]:
wordCount={}
for word in allWords:
    wordCount[word]=wordCount.get(word,0)+1

In [31]:
vocDic={w:i for i,w in enumerate([x[0] for x in wordCount.items() if x[1]>10])}

In [32]:
len(vocDic)

1848

# feature vect

I create one-hot encoding vectors to encode the features.

In [34]:
from sklearn.preprocessing import OneHotEncoder
encFirm = OneHotEncoder()
encFirm.fit([[i] for i in range(len(firmToNum))])
encInd = OneHotEncoder()
encInd.fit([[i] for i in range(len(indToNum))])
encVoc = OneHotEncoder()
encVoc.fit([[i] for i in range(len(vocDic))])
encCase = OneHotEncoder()
encCase.fit([[i] for i in range(len(caseToNum))]);

I split the data into training set (Xdata,Ydata) and testing set (xxData,yyData) in a ratio of 1 to 5.

In [37]:
Xdata=[]
Ydata=[]
xxData=[]
yyData=[]
for line in data[1:]:
    if all([l!='' for l in line]): 
        r=randint(1, 4)
        if r!=1:
            for lab in line[1].split(' | '):
                vecFirm=np.sum(
                    encFirm.transform(
                        [[firmToNum[y]] for y in line[2].split(' | ')]).toarray(),axis=0).tolist()

                vecInd=np.sum(
                    encInd.transform(
                        [[indToNum[y]] for y in line[3].split(' | ')]).toarray(),axis=0).tolist()

                vecText=np.sum(encVoc.transform([[y] for y in list(set([vocDic[stemmer.stem(w)] 
                            for w in word_tokenize(line[4].decode('utf-8')) 
                     if stemmer.stem(w) in vocDic]))]).toarray(),axis=0).tolist()

                vecTitle=np.sum(encVoc.transform([[y] for y in list(set([vocDic[stemmer.stem(w)] 
                            for w in word_tokenize(line[5].decode('utf-8')) 
                     if stemmer.stem(w) in vocDic]))]).toarray(),axis=0).tolist()

                Xdata.append(vecFirm+vecInd+vecText+vecTitle)
                Ydata.append(caseToNum[lab])
        if r==1:
            xxData.append(vecFirm+vecInd+vecText+vecTitle)
            yyData.append([caseToNum[lab] for lab in line[1].split(' | ')])

# Fitting the data

I use some simple models to predict the labels.  

In [38]:
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB

I use cross entropy to compare training and testing sets

In [39]:
def crossEnt(x1,x2):
    return -sum([x1i*math.log(x2i) for x1i,x2i in zip(x1,x2)])

Here I use a one-to-many logistic regression to make the predictions.  I calculate for different regularization parameters for L2.

In [40]:
for C in [0.001,0.01,0.1,1]:
    logistic = linear_model.LogisticRegression(C=C)
    logistic.fit(Xdata, Ydata)
    ypreds=logistic.predict_proba(xxData)
    print np.mean([crossEnt([1.0/len(yd) if y in yd else 0 for y in range(30)],ypred)
    for ypred,yd in zip(ypreds,yyData)])

2.97280671539
2.8954155611
3.43111834398
4.87573746106


Here I use a softmax logistic regression to make the predictions.  I also use different regularization parameters for L2.

In [41]:
for C in [0.001,0.01,0.1,1]:
    logistic = linear_model.LogisticRegression(C=C,multi_class='multinomial',solver='newton-cg')
    logistic.fit(Xdata, Ydata)
    ypreds=logistic.predict_proba(xxData)
    print np.mean([crossEnt([1.0/len(yd) if y in yd else 0 for y in range(30)],ypred)
    for ypred,yd in zip(ypreds,yyData)])

2.90441085009
2.91450462128
3.58556891096
5.11802789023


Finally, I use a multinomial naive bayes model with different smoothing values.

In [49]:
for alpha in [1,200.0,1000.0,10000.0]:
    model=MultinomialNB(alpha=alpha)
    model.fit(Xdata,Ydata)
    ypreds=model.predict_proba(xxData)
    print np.mean([crossEnt([1.0/len(yd) if y in yd else 0 for y in range(30)],ypred)
        for ypred,yd in zip(ypreds,yyData)])

24.1728369428
5.01252423929
3.12039078177
2.92447850754


# Conclusions

First we can not that the value of cross entropy of randomly choosing a label would be:

In [50]:
-math.log(1/30.0)

3.4011973816621555

So the models do seem to be capturing some information.  But probably not as well as one would hope.  

## Future work

Here I used a simple 1-gram one-hot encoding model for the text data.  I could have use other models such a 2-gram, TFIDF, glove, or w2v.  But for the sake of time just went with the simpler model.  Exploring other embeddings would definitely be a good idea.  

Additionally, I did not at all attempt to do any feature engineering.  I could have done this to improve the predictions.  I could have also used a neural network which does automatic feature selection.  

# Additional Questions

### What is part of speech (POS) tagging? What is the simplest approach to building a POS tagger that you can imagine?

A part of speech tagger finds the part of speech of individual words in a sentence.  The simplest part of speech would simply assign the most common part of speech to a given term, ignore all other information from the surrounding words.

### How would you build a POS tagger from scratch given a corpus of annotated sentences? How would you deal with unknown words?

I would use a linear chain conditional random field model since this is one of the best yet simplest part of speech taggers and would be easily trained.  Unknown words can still be labeled using a linear chain conditional random field model because there is information about the words that come before and after the given word.

### How would you train a model that identifies whether the word “Apple” in a sentence belongs to the fruit or the company?

I would train a W2V model on a large corpus such a Wikipedia.  I would then calculate the W2V for the article apple as a fruit and apple as the company.  I would then calculate the cosine distance between the concept W2v and the W2V of the document that I am trying to disambiguate. 

### How would you find all the occurrences of quoted text in a news article?

First, I would tokenize the sentene.  I would scan the sentence for a beginning quote and then for an ending quote.  If I find both then it is likely a quote.

### How would you build a system that auto corrects text that has been generated by a speech recognition system?

We could compare the word to similar words that are with a dictionary.  If the words are similar to one another and one is more commonly used that the other, then we might make the correction.  A more sophisticated system might also compare other words in the sentence and calculate the likelihood of that word given the other words in the sentence.

### What is latent semantic indexing and where can it be applied?

LSI transforms documents and terms into 'concepts' via an SVD decompostion.  Documents in a corpus are mathematically represented in a document vs. term matrix and this matrix is then decomposed using SVD.  It is useful for understanding similarities between terms and documents in terms of co-ocurrences of 

### How would you build a system that automatically groups news articles by subject?

I would first train a LDA topic model.  I would then classify the new articles according to this model.

### What are stop words? Describe an application in which stop words should be removed.

Stop words are very common words that help to make the grammar of a sentence correct but are often to common to be useful for common NLP tasks and thus are often removed before processing.  The words should be removed when they contain no signal that would help in the task at hand and would just create larger feature vectors and add noise.

### What is entropy? How would you estimate the entropy of the English language?

Entropy is the amount of information in data.  If we have an estimate of the probability of words in the English language we can calculate entropy using the typical formula.  According to wikipedia, English text has between 0.6 and 1.3 bits of entropy per character of the message.

### What is the TF-IDF score of a word and in what context is this useful?

TF-IDF stands for term frequency, inverse document frequency and calculates just that --- the frequency of a term times the inverse document frequency (often with slight modification).  This is useful as a vectorization of words as features as it gives greater weight to less common words.  It is also useful for understand how descriptive a given word is.  Low score indicate a common word while high scores indicate a word with a very specific meaning.
